In [85]:
import pandas as pd
from datetime import datetime
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

DATA_FILE = 'data/list_of_cars.csv'


def preprocess(data_file):
    df = pd.read_csv(data_file,header=None, names=['Brand', 'Body', 'Model', 'Year', 'Price', 'HP', 'Torque', 'Acc', 'Braking', 'Road Holding', 'Fuel Economy'])
    df['HP'] = df['HP'].apply(lambda x: str(x)[:3].strip()).astype(float).dropna()
    df['Torque'] = df['Torque'].apply(lambda x: str(x)[:3].strip()).astype(float).dropna()
    df['Braking'] = df['Braking'].apply(lambda x: str(x)[:3].strip()).astype(float).dropna()
    return df

def calculate_score(row, average_score, max_score, attribute):
    
    final_score = 0
    
    if row[attribute] >= max_score:
        return 5
    else:
        calculate_difference = row[attribute]/average_score
        final_score = calculate_difference * 2.5
        
    return final_score

data = preprocess(DATA_FILE)

avg_HP = data['HP'].mean()
max_HP = data['HP'].max()
avg_torque = data['Torque'].mean()
max_torque = data['Torque'].max()
avg_braking_distance = data['Braking'].mean()
max_braking_distance = data['Braking'].max()

d = {
    'Brand': data['Brand'], 
    'Body': data['Body'], 
    'Year': data['Year'],
    'Price': data['Price']
}

hp_score_list = []
torque_score_list = []
braking_score_list = []

for index, row in data.iterrows():
    calculated_HP_score = calculate_score(row, avg_HP, max_HP, 'HP')
    hp_score_list.append(calculated_HP_score)
    calculated_torque_score = calculate_score(row, avg_torque, max_torque, 'Torque')
    torque_score_list.append(calculated_torque_score)
    calculated_braking_score = calculate_score(row, avg_braking_distance, max_braking_distance, 'Braking')
    braking_score_list.append(calculated_braking_score)
    
df = pd.DataFrame(data=d)

df['HP'] = hp_score_list
df['Torque'] =  torque_score_list
df['Braking'] = braking_score_list
print(df)

#df.to_csv('data/model_vectors.csv')


        

         Body                    Brand   Price    Year        HP    Torque  \
0       Sedan                     Audi   51800  2017.0  3.013777  3.162651   
1       Sedan                      BMW   45900  2017.0  2.965940  2.988879   
2       Sedan                 Infinity   39900  2018.0       NaN       NaN   
3       Sedan                    Lexus   41050  2017.0  3.659587  3.208990   
4       Sedan                      BMW   56850  2018.0  3.827019  3.822984   
5       Sedan                 Mercedes   44700  2018.0  2.882223  3.162651   
6         SUV                    Volvo   59150  2018.0       NaN       NaN   
7         SUV                    Lexus   56500  2017.0       NaN       NaN   
8         SUV                      BMW   69950  2016.0       NaN       NaN   
9         SUV                     Audi   74750  2017.0  3.013777  3.162651   
10        SUV            Mercedes Benz   50300  2017.0  2.882223  3.162651   
11        SUV              Range Rover   62000  2018.0  5.000000